In [2]:
! pip install faiss-cpu==1.7.4 mistralai


  Using cached faiss-cpu-1.7.4.tar.gz (57 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build faiss-cpu


  error: subprocess-exited-with-error
  
  × Building wheel for faiss-cpu (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      C:\Users\priya\AppData\Local\Temp\pip-build-env-7v2nsem0\overlay\Lib\site-packages\setuptools\dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license expression:
      
              License :: OSI Approved :: MIT License
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              ********************************************************************************
      
      !!
        self._finalize_license_expression()
      running bdist_wheel
      running build
      running build_py
      running build_ext
      building 'faiss

In [4]:
from mistralai.client import MistralClient
from getpass import getpass
import requests
import numpy as np

api_key = getpass("Type your API Key")
client = MistralClient(api_key=api_key)
     

In [5]:
response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

In [6]:
f = open('essay.txt','w')
f.write(text)
f.close()

In [7]:
len(text)

75014

In [8]:
chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0,len(text), chunk_size)]
len(chunks)

37

In [9]:
def get_embedding(input):
    embeddings_batch_response = client.embeddings.create(
          model="mistral-embed",
          inputs=input
      )
    return embeddings_batch_response.data[0].embedding

In [11]:
text_embeddings = np.array([get_embedding(chunk) for chunk in chunks])

AttributeError: 'function' object has no attribute 'create'

In [12]:
text_embeddings.shape

NameError: name 'text_embeddings' is not defined

In [ ]:
text_embeddings

array([[-0.03979492,  0.07733154,  0.00013709, ..., -0.01274109,
        -0.02101135, -0.00264168],
       [-0.03152466,  0.07226562,  0.02961731, ..., -0.01079559,
        -0.01189423, -0.00821686],
       [-0.05905151,  0.06112671,  0.01206207, ..., -0.0226593 ,
         0.00488663, -0.00665283],
       ...,
       [-0.05477905,  0.06890869,  0.02703857, ..., -0.02456665,
        -0.02526855, -0.02687073],
       [-0.03884888,  0.05587769,  0.04718018, ..., -0.01812744,
         0.00926208, -0.00866699],
       [-0.03048706,  0.05831909,  0.01704407, ..., -0.01620483,
        -0.01800537, -0.04415894]])

In [ ]:
d = text_embeddings.shape[1]
Index = faiss.IndexFlatL2(d)
Index.add(text_embeddings)

In [ ]:
question = "What were the two main things the author worked on before college?"
question_embeddings = np.array([get_embeddings(question)])
question_embeddings.shape

(1, 1024)

In [ ]:
question_embeddings

array([[-0.05447388,  0.03479004,  0.0375061 , ..., -0.02787781,
        -0.00327492,  0.0029068 ]])

In [ ]:
D, I = Index.search(question_embeddings, k =2)
print(I)

[[0 3]]


In [ ]:
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
print(retrieved_chunk)

['\n\nWhat I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.\n\nThe language we used was an early version of Fortran. You had to type programs on punch cards, then 

In [ ]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

def run_mistral(user_message, model = "mistral-large-latest"):
    messages = [
        {
            "role" : "user", "content" : user_message
        }
    ]
    chat_response = client.chat.complete(
        model = model,
        messages = messages
    )
    return (chat_response.choices[0].message.content)


In [ ]:
run_mistral(prompt)

'The two main things the author worked on before college were writing and programming.'

In [13]:
from langchain_community.document_loaders import TextLoader
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

load = TextLoader("essay.txt")
docs = load.load()

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

embeddings = MistralAIEmbeddings(model = "mistral-embed", mistral_api_key = api_key)

vector = FAISS.from_documents(documents, embeddings)

retriever = vector.as_retriever()

model = ChatMistralAI(mistral_api_key = api_key)

prompt = ChatPromptTemplate.from_template(""" Answer the following question based only on the provided context:


{context}


Question: {input}

""")

document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever,document_chain)
response = retrieval_chain.invoke({"input": "What were the two main things the author worked on before college?"})
print(response["answer"])

c:\Users\priya\anaconda3\Lib\site-packages\langchain_mistralai\embeddings.py:186: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


The two main things the author worked on before college were writing and programming. He wrote short stories and tried writing programs on an IBM 1401 using an early version of Fortran.


In [ ]:
!pip install llama-index==0.10.55 llama-index-llms-mistralai==0.1.18 llama-index-embeddings-mistralai mistralai==0.4.2
     

  Using cached llama_index_readers_llama_parse-0.4.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached Deprecated-1.2.18-py2.py3-none-any.whl.metadata (5.7 kB)
  Using cached dirtyjson-1.0.8-py3-none-any.whl.metadata (11 kB)
  Using cached tiktoken-0.9.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
INFO: pip is looking at multiple versions of llama-index-embeddings-mistralai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
  Using cached llama_index_indices_managed_llama_cloud-0.6.11-py3-none-any.whl.metadata (3.6 kB)
INFO: This is taking longer than usual. You might need to provide the depende

In [14]:
#llamaindex

import os
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding


reader = SimpleDirectoryReader(input_files=['essay.txt'])
documents = reader.load_data()

Settings.llm = MistralAI(model="mistral-medium", api_key= api_key)
Settings.embed_model = MistralAIEmbedding(model_name="mistral-embed", api_key=api_key)

Index = VectorStoreIndex.from_documents(documents)

query_engine = Index.as_query_engine(similarity_top_k = 2)
reponse = query_engine.query(
    "What were the two main things the author worked on before college?"
)
print(str(response))


{'input': 'What were the two main things the author worked on before college?', 'context': [Document(id='229a0367-5c56-4faa-bf03-af13cfa5f9c3', metadata={'source': 'essay.txt'}, page_content='What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, pr